<a href="https://colab.research.google.com/github/songjinu/test_flow/blob/main/Auto_Evaluate_v2_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Set Environment

In [ ]:
!pip install mlflow
!pip install langchain
!pip install langchain-openai
!pip install langchain-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.0/552.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7

In [ ]:
import logging
from collections import Counter

import os
import mlflow
import pandas as pd

from tqdm import tqdm
from tqdm.contrib import tzip
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

In [ ]:
logger = logging.getLogger(__name__)
"""
Open AI API Key, Antropic API Key를 입력해주세요.
"""
os.environ["OPENAI_API_KEY"] = ""
os.environ["ANTHROPIC_API_KEY"] = ""

In [ ]:
# TONIC에서 생성된 답변과 기준 답변 간의 유사성을 비교하는 프롬프트 템플릿
TONIC_ANSWER_SIMILARITY_PROMPT = (
    "Considering the reference answer and the new answer to the following question, "
    "on a scale of 0 to 5, where 5 means the same and 0 means not at all similar, "
    "how similar in meaning is the new answer to the reference answer? Respond with just "
    "a number and no additional text.\nQUESTION: {question}\nREFERENCE ANSWER: {"
    "reference_answer}\nNEW ANSWER: {llm_answer}\n"
)

# ALLGANIZE에서 생성된 답변이 기준 답변과 일치하는지 확인하는 프롬프트 템플릿
ALLGANIZE_ANSWER_CORRECTNESS_PROMPT = """
question = \"\"\"
{question}
\"\"\"

target_answer = \"\"\"
{reference_answer}
\"\"\"

generated_answer = \"\"\"
{llm_answer}
\"\"\"

Check if target_answer and generated_answer match by referring to question.
If target_answer and generated_answer match 1, answer 0 if they do not match.
Only 1 or 0 must be created.
"""


def tonic_validate(questions: list, generated_answers: list, target_answers: list, model: str) -> list:
    """
    생성된 답변과 기준 답변 간의 유사성을 특정 LLM을 사용하여 검증합니다.

    Args:
        questions (list): 질문 목록.
        generated_answers (list): 모델이 생성한 답변 목록.
        target_answers (list): 기준(정답) 답변 목록.
        model (str): 사용할 LLM 모델 이름.

    Returns:
        list: 각 답변 비교에 대한 유사성 점수 목록 (0 ~ 5).
    """
    llm = ChatOpenAI(model_name=model)

    prompt = PromptTemplate(
        input_variables=["question", "reference_answer", "llm_answer"], template=TONIC_ANSWER_SIMILARITY_PROMPT
    )
    chain = LLMChain(llm=llm, prompt=prompt)

    eval_check = []
    for question, target_answer, generated_answer in zip(tqdm(questions), target_answers, generated_answers):
        try:
            # LLM 체인을 실행하여 유사성 점수를 얻음
            llm_result = chain.run(
                {"question": question, "reference_answer": target_answer, "llm_answer": generated_answer}
            )
            eval_check.append(int(llm_result))
        except Exception as e:
            logger.warning(f"llm_eval exception: {e}")
            eval_check.append(-1)
    return eval_check


def mlflow_eval(question_list: list, answer_list: list, ground_truth_list: list, model: str):
    """
    MLflow 메트릭을 사용하여 답변의 유사성과 정확성을 평가합니다.

    Args:
        question_list (list): 질문 목록.
        answer_list (list): 생성된 답변 목록.
        ground_truth_list (list): 기준 답변 목록(정답).
        model (str): MLflow 평가에 사용할 모델 이름.

    Returns:
        tuple: MLflow 평가에서 나온 유사성 및 정확성 점수.
    """
    eval_data = pd.DataFrame({"inputs": question_list, "predictions": answer_list, "ground_truth": ground_truth_list})

    with mlflow.start_run():
        # 사용자 정의 메트릭을 사용하여 MLflow 평가 실행
        results = mlflow.evaluate(
            data=eval_data,
            targets="ground_truth",
            predictions="predictions",
            extra_metrics=[
                mlflow.metrics.genai.answer_similarity(model=model),
                mlflow.metrics.genai.answer_correctness(model=model),
            ],
            evaluators="default",
        )

        eval_table = results.tables["eval_results_table"]
        mlflow_answer_similarity = eval_table["answer_similarity/v1/score"].tolist()
        mlflow_answer_correctness = eval_table["answer_correctness/v1/score"].tolist()

    return mlflow_answer_similarity, mlflow_answer_correctness


def allganize_eval(
    questions: list, generated_answers: list, target_answers: list, model: str
) -> list:
    """
    생성된 답변의 정확성을 기준 답변과 비교하여 검증합니다.

    Args:
        questions (list): 질문 목록.
        generated_answers (list): 모델이 생성한 답변 목록.
        target_answers (list): 기준(정답) 답변 목록.
        model (str): 사용할 LLM 모델 이름.

    Returns:
        list: 정확성 점수 목록 (정확하면 1, 틀리면 0).
    """
    llm = ChatAnthropic(model=model)

    prompt = PromptTemplate(
        input_variables=["question", "reference_answer", "llm_answer", "contexts"],
        template=ALLGANIZE_ANSWER_CORRECTNESS_PROMPT,
    )
    chain = LLMChain(llm=llm, prompt=prompt)

    eval_check = []
    for question, target_answer, generated_answer in zip(tqdm(questions), target_answers, generated_answers):
        try:
            # LLM 체인을 실행하여 정확성 점수를 얻음
            llm_result = chain.run(
                {
                    "question": question,
                    "reference_answer": target_answer,
                    "llm_answer": generated_answer,
                }
            )
            eval_check.append(int(llm_result))
        except Exception as e:
            logger.warning(f"llm_eval exception: {e}")
            eval_check.append(-1)

    return eval_check


def most_frequent_element(result: list) -> str:
    """
    리스트에서 가장 빈번하게 등장하는 요소를 반환하며, 특정 값에 우선순위를 둡니다.

    Args:
        result (list): 평가 결과 목록.

    Returns:
        str: 가장 빈번한 결과 ("O"는 정답, "X"는 오답).
    """
    count = Counter(result)
    priority = ["X", "O"]  # 'X'와 'O'에 대한 우선순위 정의

    most_common = count.most_common()
    for element in priority:
        if element in count and count[element] == most_common[0][1]:
            return element


def get_evaluation_result(score: int) -> str:
    """
    평가 점수를 'O'(정답) 또는 'X'(오답)로 분류합니다.

    Args:
        score (int): 분류할 점수.

    Returns:
        str: 점수가 4 이상이면 'O', 아니면 'X'.
    """
    if score >= 4:
        return "O"
    else:
        return "X"


def eval_vote(
    tonic_answer_similarity: list,
    mlflow_answer_similarity: list,
    mlflow_answer_correctness: list,
    allganize_answer_correctness: list,
) -> list:
    """
    여러 출처의 평가 결과를 종합하여 최종 평가를 결정합니다.

    Args:
        tonic_answer_similarity (list): LLM 유사성 검증 결과 점수.
        mlflow_answer_similarity (list): MLflow에서 나온 유사성 점수.
        mlflow_answer_correctness (list): MLflow에서 나온 정확성 점수.
        allganize_answer_correctness (list): LLM 정확성 검증 결과 점수.

    Returns:
        list: 각 답변에 대한 최종 평가 결과 ('O'는 정답, 'X'는 오답).
    """
    e2e_result = []
    for i in range(len(tonic_answer_similarity)):
        tonic_answer_similarity_ox = get_evaluation_result(tonic_answer_similarity[i])
        mlflow_answer_similarity_ox = get_evaluation_result(mlflow_answer_similarity[i])
        mlflow_answer_correctness_ox = get_evaluation_result(mlflow_answer_correctness[i])
        allganize_answer_correctness_ox = "O" if allganize_answer_correctness[i] == 1 else "X"

        # 가장 빈번한 평가 결과를 사용하여 종합 결과 결정
        e2e_result.append(
            most_frequent_element(
                [
                    tonic_answer_similarity_ox,
                    mlflow_answer_similarity_ox,
                    mlflow_answer_correctness_ox,
                    allganize_answer_correctness_ox,
                ]
            )
        )
    return e2e_result


def llm_evaluate(question: list, generated_answer: list, target_answer: list) -> list:
    """
    여러 평가 방법을 사용하여 LLM이 생성한 답변을 종합 평가합니다.

    Args:
        question (list): 질문 목록.
        generated_answer (list): 모델이 생성한 답변 목록.
        target_answer (list): 기준 답변 목록.

    Returns:
        list: 각 답변에 대한 최종 평가 결과 ('O'는 정답, 'X'는 오답).
    """
    tonic_answer_similarity = tonic_validate(question, generated_answer, target_answer, model="gpt-4-turbo")
    mlflow_answer_similarity, mlflow_answer_correctness = mlflow_eval(question, generated_answer, target_answer, model="openai:/gpt-4o")
    allganize_answer_correctness = allganize_eval(question, generated_answer, target_answer, model="claude-3-opus-20240229")

    # 모든 평가 출처로부터 결과를 종합
    return eval_vote(tonic_answer_similarity, mlflow_answer_correctness, mlflow_answer_similarity, allganize_answer_correctness)

In [ ]:
if __name__ == "__main__":
    question = ["2024년 1월, 2월, 3월 각각의 평균 조달금리와 응찰률이 어떻게 되나요?", "2024년 1월, 2월, 3월 각각의 평균 조달금리와 응찰률이 어떻게 되나요?"]
    generated_answer = ["2024년 1월의 평균 조달금리는 3.27%, 응찰률은 333%입니다. 2월의 평균 조달금리는 3.36%, 응찰률은 335%입니다. 3월의 평균 조달금리는 3.32%, 응찰률은 334%입니다[2].", "2024년 1월, 2월, 3월의 평균 조달 금리는 각각 3.57%, 3.52%, 3.32% 입니다. 응찰률은 각각 271%, 285%, 334% 입니다."]
    target_answer = ["2024년 1월의 평균 조달금리는 3.27%, 응찰률은 333이며, 2월의 평균 조달금리는 3.36%, 응찰률은 335이며, 3월의 평균 조달금리는 3.32%, 응찰률은 334입니다.", "2024년 1월의 평균 조달금리는 3.27%, 응찰률은 333%입니다. 2월의 평균 조달금리는 3.36%, 응찰률은 335%입니다. 3월의 평균 조달금리는 3.32%, 응찰률은 334%입니다[2]."]

    result = llm_evaluate(question, generated_answer, target_answer)
    print("\n\n", result)

100%|██████████| 2/2 [00:00<00:00,  2.32it/s]
2024/09/06 04:13:22 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  7.62it/s]



 ['O', 'X']
